In [22]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd


# librerias de soporte 
# ------------------------------------------------------------------------------
import sys
sys.path.append('../src')
import suport as sp



# Modelado y evaluación
# ------------------------------------------------------------------------------
from pycaret.regression import *
from pycaret.regression import evaluate_model

## opciones de pandas para visualizar todas las columnas 
import warnings
warnings.simplefilter('ignore')

In [23]:
df = pd.read_csv('../data/Customers_cluster_weekend.csv') # leemos el archivo csv


df.head()

,id,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,Cluster
0,0,BU79786,Washington,2763.52,No,Basic,Bachelor,2/24/11,Employed,F,...,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.81,Two-Door Car,Medsize,2
1,1,QZ44356,Arizona,6979.54,No,Extended,Bachelor,1/31/11,Unemployed,F,...,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.46,Four-Door Car,Medsize,1
2,2,AI49188,Nevada,12887.43,No,Premium,Bachelor,2/19/11,Employed,F,...,0,2,Personal Auto,Personal L3,Offer1,Agent,566.47,Two-Door Car,Medsize,2
3,3,WW63253,California,7645.86,No,Basic,Bachelor,1/20/11,Unemployed,M,...,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.88,SUV,Medsize,1
4,4,HB64268,Washington,2813.69,No,Basic,Bachelor,2/3/11,Employed,M,...,0,1,Personal Auto,Personal L1,Offer1,Agent,138.13,Four-Door Car,Medsize,2


In [24]:
objetivo = 'customer_lifetime_value'
numericas  = list(df.select_dtypes(include = np.number ).columns.drop(objetivo))
categoricas = list(df.select_dtypes(exclude = 'number').columns)
ignorar = []
print(numericas)
print(categoricas)

['id', 'income', 'monthly_premium_auto', 'months_since_last_claim', 'months_since_policy_inception', 'number_of_open_complaints', 'number_of_policies', 'total_claim_amount', 'Cluster']
['customer', 'state', 'response', 'coverage', 'education', 'effective_to_date', 'employmentstatus', 'gender', 'location_code', 'marital_status', 'policy_type', 'policy', 'renew_offer_type', 'sales_channel', 'vehicle_class', 'vehicle_size']


In [25]:
exp1 = RegressionExperiment()
exp1.setup(df, ignore_features=ignorar, target=objetivo, 
            session_id=42, train_size=0.7, 
            categorical_features=categoricas, numeric_features=numericas)

,Description,Value
0,Session id,42
1,Target,customer_lifetime_value
2,Target type,Regression
3,Original data shape,"(9134, 26)"
4,Transformed data shape,"(9134, 67)"
5,Transformed train set shape,"(6393, 67)"
6,Transformed test set shape,"(2741, 67)"
7,Numeric features,9
8,Categorical features,16
9,Preprocess,True


In [26]:
# get the most successful top 3 models
top3_exp1 = exp1.compare_models(n_select=1)
top3_exp1

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,4107.0329,42576966.4246,6509.3760,0.0560,0.6474,0.6729,0.6780
lightgbm,Light Gradient Boosting Machine,4347.3818,45051190.3336,6697.0425,0.0005,0.6860,0.7406,0.2660
llar,Lasso Least Angle Regression,4351.7685,45139998.7364,6703.7156,-0.0015,0.6865,0.7410,0.0430
dt,Decision Tree Regressor,4351.6649,45139823.8694,6703.6999,-0.0015,0.6865,0.7410,0.0620
lr,Linear Regression,4351.7685,45139998.7364,6703.7156,-0.0015,0.6865,0.7410,0.2480
omp,Orthogonal Matching Pursuit,4351.7685,45139998.7364,6703.7156,-0.0015,0.6865,0.7410,0.0490
rf,Random Forest Regressor,4352.0277,45140778.0919,6703.7694,-0.0015,0.6866,0.7411,0.6410
lar,Least Angle Regression,4351.7685,45139998.7364,6703.7156,-0.0015,0.6865,0.7410,0.0410
en,Elastic Net,4351.7685,45139999.0253,6703.7157,-0.0015,0.6865,0.7410,0.0420
ridge,Ridge Regression,4351.7685,45139998.7305,6703.7156,-0.0015,0.6865,0.7410,0.0410


ExtraTreesRegressor(n_jobs=-1, random_state=42)

In [27]:
exp1 = RegressionExperiment()
exp1.setup(df[:-1], ignore_features=ignorar, target=objetivo, 
            session_id=42, train_size=0.7, 
            categorical_features=categoricas, numeric_features=numericas)

# get the most successful top 3 models
top3_exp1 = exp1.compare_models(n_select=1)
modelo_final = exp1.create_model(top3_exp1)


'''
# Evaluar el modelo y obtener el RMSE
rmse = evaluate_model(modelo_final)['RMSE']

print("RMSE del modelo_final:", rmse)
modelo_final

'''

,Description,Value
0,Session id,42
1,Target,customer_lifetime_value
2,Target type,Regression
3,Original data shape,"(9133, 26)"
4,Transformed data shape,"(9133, 67)"
5,Transformed train set shape,"(6393, 67)"
6,Transformed test set shape,"(2740, 67)"
7,Numeric features,9
8,Categorical features,16
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,4099.9223,43263627.8928,6553.0400,0.0566,0.6471,0.6720,0.5880
lightgbm,Light Gradient Boosting Machine,4339.4690,45822720.1329,6746.4520,-0.0005,0.6853,0.7374,0.2440
gbr,Gradient Boosting Regressor,4356.7681,45881584.8510,6750.9899,-0.0019,0.6876,0.7439,0.2760
lar,Least Angle Regression,4342.7216,45889375.8870,6751.3751,-0.0020,0.6857,0.7381,0.0380
omp,Orthogonal Matching Pursuit,4342.7216,45889375.8870,6751.3751,-0.0020,0.6857,0.7381,0.0370
rf,Random Forest Regressor,4342.6872,45889124.0894,6751.3648,-0.0020,0.6857,0.7381,0.5580
llar,Lasso Least Angle Regression,4342.7216,45889375.8870,6751.3751,-0.0020,0.6857,0.7381,0.0390
dt,Decision Tree Regressor,4343.5098,45890189.9422,6751.4377,-0.0020,0.6858,0.7384,0.0460
en,Elastic Net,4342.7217,45889376.6091,6751.3751,-0.0020,0.6857,0.7381,0.0380
ridge,Ridge Regression,4342.7216,45889375.8810,6751.3751,-0.0020,0.6857,0.7381,0.0390


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4078.7263,41732145.4809,6460.0422,0.0509,0.6555,0.6970
1,4155.6134,42061736.4252,6485.5020,0.0600,0.6419,0.6532
2,3864.7909,33945930.2658,5826.3136,0.0682,0.6532,0.7196
3,4477.1927,58586500.8139,7654.1819,0.0372,0.6654,0.6563
4,3885.6005,40848989.5550,6391.3214,0.0697,0.6254,0.6397
5,4305.7504,53226565.0123,7295.6538,0.0475,0.6690,0.6963
6,3826.3843,31744555.5899,5634.2307,0.0634,0.6196,0.6512
7,4244.9992,44503251.0644,6671.0757,0.0557,0.6559,0.6836
8,4060.9204,44408546.5253,6663.9738,0.0610,0.6294,0.6321


'\n# Evaluar el modelo y obtener el RMSE\nrmse = evaluate_model(modelo_final)[\'RMSE\']\n\nprint("RMSE del modelo_final:", rmse)\nmodelo_final\n\n'

## CAsi final 42

In [29]:
# definimos conjuntos de entrenamiento y test
df_train = df[:-1]
df_test = df[-1:]

#n generamos objeto de experimento
exp1 = RegressionExperiment()
exp1.setup(df_train, ignore_features=ignorar, target=objetivo, 
            session_id=42, train_size=0.7, 
            categorical_features=categoricas, numeric_features=numericas)


# seleccionamos el mejor modelo 
top3_exp1 = exp1.compare_models(n_select=1)

# entrenamos el modelo 
modelo_final = exp1.create_model(top3_exp1)

,Description,Value
0,Session id,42
1,Target,customer_lifetime_value
2,Target type,Regression
3,Original data shape,"(9133, 26)"
4,Transformed data shape,"(9133, 67)"
5,Transformed train set shape,"(6393, 67)"
6,Transformed test set shape,"(2740, 67)"
7,Numeric features,9
8,Categorical features,16
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,4099.9223,43263627.8928,6553.0400,0.0566,0.6471,0.6720,0.6050
lightgbm,Light Gradient Boosting Machine,4339.4690,45822720.1329,6746.4520,-0.0005,0.6853,0.7374,0.2510
gbr,Gradient Boosting Regressor,4356.7681,45881584.8510,6750.9899,-0.0019,0.6876,0.7439,0.2780
lar,Least Angle Regression,4342.7216,45889375.8870,6751.3751,-0.0020,0.6857,0.7381,0.0360
omp,Orthogonal Matching Pursuit,4342.7216,45889375.8870,6751.3751,-0.0020,0.6857,0.7381,0.0350
rf,Random Forest Regressor,4342.6872,45889124.0894,6751.3648,-0.0020,0.6857,0.7381,0.5650
llar,Lasso Least Angle Regression,4342.7216,45889375.8870,6751.3751,-0.0020,0.6857,0.7381,0.0340
dt,Decision Tree Regressor,4343.5098,45890189.9422,6751.4377,-0.0020,0.6858,0.7384,0.0450
en,Elastic Net,4342.7217,45889376.6091,6751.3751,-0.0020,0.6857,0.7381,0.0350
ridge,Ridge Regression,4342.7216,45889375.8810,6751.3751,-0.0020,0.6857,0.7381,0.0360


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4078.7263,41732145.4809,6460.0422,0.0509,0.6555,0.6970
1,4155.6134,42061736.4252,6485.5020,0.0600,0.6419,0.6532
2,3864.7909,33945930.2658,5826.3136,0.0682,0.6532,0.7196
3,4477.1927,58586500.8139,7654.1819,0.0372,0.6654,0.6563
4,3885.6005,40848989.5550,6391.3214,0.0697,0.6254,0.6397
5,4305.7504,53226565.0123,7295.6538,0.0475,0.6690,0.6963
6,3826.3843,31744555.5899,5634.2307,0.0634,0.6196,0.6512
7,4244.9992,44503251.0644,6671.0757,0.0557,0.6559,0.6836
8,4060.9204,44408546.5253,6663.9738,0.0610,0.6294,0.6321


In [30]:
# guardamos el modelo 
exp1.save_model(modelo_final, 'my_pycaret_regression')


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['id', 'income',
                                              'monthly_premium_auto',
                                              'months_since_last_claim',
                                              'months_since_policy_inception',
                                              'number_of_open_complaints',
                                              'number_of_policies',
                                              'total_claim_amount', 'Cluster'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=...
                                                                     'renew_offer_type',
                                                                     'sales_channel',
                                                                     'vehicle_cla

In [38]:
# cargamos el modelo
my_winning_regressor = load_model('my_pycaret_regression')


Transformation Pipeline and Model Successfully Loaded


In [41]:
pred = predict_model(my_winning_regressor, data=df_train)
result = pull()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,1301.0695,14337335.5640,3786.4674,0.6963,0.3672,0.2094


In [40]:
result

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,1301.4487,1.433825e+07,3786.5883,0.6963,0.3673,0.2096


--------------

In [9]:
boston_train = boston[:-1]
boston_test = boston[-1:]

In [43]:
from pycaret.datasets import get_data
boston = get_data('boston')
from pycaret.regression import *
exp_name = setup(data = boston_train,  target = 'medv')


,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


,Description,Value
0,Session id,6845
1,Target,medv
2,Target type,Regression
3,Original data shape,"(505, 14)"
4,Transformed data shape,"(505, 14)"
5,Transformed train set shape,"(353, 14)"
6,Transformed test set shape,"(152, 14)"
7,Numeric features,13
8,Preprocess,True
9,Imputation type,simple


In [11]:
lr = create_model('lr')
lr

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,2.2815,9.6761,3.1106,0.8063,0.1182,0.1012
1,3.9350,28.5828,5.3463,0.7741,0.1953,0.1513
2,3.8116,27.5850,5.2521,0.6133,0.2356,0.2337
3,3.9658,29.3304,5.4158,0.7217,0.3550,0.2270
4,2.8378,15.3350,3.9160,0.8408,0.2179,0.1620
5,4.3457,58.6782,7.6602,0.5040,0.2867,0.2013
6,3.9353,34.0724,5.8372,0.6845,0.2420,0.1756
7,3.0760,17.2949,4.1587,0.7795,0.3153,0.1642
8,3.6396,23.6730,4.8655,0.7392,0.1839,0.1592


LinearRegression(n_jobs=-1)

In [12]:
pred = predict_model(lr, data=boston_test)
result = pull()


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,10.8767,118.3030,10.8767,nan,0.6115,0.9140


In [ ]:
resultpred = predict_model(my_winning_regressor, data=df_test)
result = pull()

In [61]:
from pycaret.datasets import get_data
from pycaret.regression import *

# Cargar un conjunto de datos de ejemplo
data = df_train

# Configurar el entorno de PyCaret
s = setup(data, target='customer_lifetime_value')

# Crear un modelo Random Forest para regresión
model_rf = create_model('rf')

# Hacer predicciones en el conjunto de datos
predictions = predict_model(model_rf, data=data)

result = pull()
print(result['RMSE'][0])

# Mostrar las predicciones y la columna 'customer_lifetime_value'
print(predictions[['customer_lifetime_value']])

,Description,Value
0,Session id,4991
1,Target,customer_lifetime_value
2,Target type,Regression
3,Original data shape,"(9133, 26)"
4,Transformed data shape,"(9133, 67)"
5,Transformed train set shape,"(6393, 67)"
6,Transformed test set shape,"(2740, 67)"
7,Numeric features,9
8,Categorical features,16
9,Preprocess,True


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4264.8018,44191726.9971,6647.6858,-0.0019,0.6813,0.7442
1,4705.3744,56990404.4200,7549.1989,-0.0039,0.7102,0.7441
2,4510.0973,45714198.3373,6761.2276,-0.0002,0.7124,0.7890
3,4742.9846,56646396.1443,7526.3800,-0.0018,0.7139,0.7629
4,4315.5216,43902469.3208,6625.8939,-0.0000,0.6909,0.7526
5,4145.1666,35939513.3826,5994.9573,-0.0018,0.6718,0.7319
6,4412.8443,44101334.9400,6640.8836,-0.0000,0.6912,0.7546
7,4567.6329,46561852.0903,6823.6246,-0.0025,0.6984,0.7463
8,4146.7830,36589667.6284,6048.9394,-0.0061,0.6879,0.7814


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,1327.1762,14457615.8913,3802.3172,0.6937,0.3790,0.2241


3802.3172
      customer_lifetime_value
0                 2763.520020
1                 6979.540039
2                12887.429688
3                 7645.859863
4                 2813.689941
...                       ...
9128              4100.399902
9129             23405.990234
9130              3096.510010
9131              8163.890137
9132              7524.439941

[9133 rows x 1 columns]


In [59]:
# guardamos el modelo 
s.save_model(model_rf, 'Random_forest')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['id', 'income',
                                              'monthly_premium_auto',
                                              'months_since_last_claim',
                                              'months_since_policy_inception',
                                              'number_of_open_complaints',
                                              'number_of_policies',
                                              'total_claim_amount', 'Cluster'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=...
                                                                     'renew_offer_type',
                                                                     'sales_channel',
                                                                     'vehicle_cla

In [51]:
rf_carga= load_model('Random_forest')


Transformation Pipeline and Model Successfully Loaded


In [56]:
pred = predict_model(my_winning_regressor, data=df_test)
result = pull()

In [54]:
df_test.drop('customer_lifetime_value', axis=1, inplace=True)

In [55]:
df_test

,id,customer,state,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,Cluster
9133,9133,Y167826,California,No,Extended,College,2/14/11,Unemployed,M,0,...,0,1,Corporate Auto,Corporate L3,Offer4,Call Center,369.6,Two-Door Car,Medsize,1


In [57]:
result

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,4764.4631,2.270011e+07,4764.4631,NaN,1.038,1.8242


In [ ]:

            # generamos objeto de experimento
            exp1 = RegressionExperiment()
            exp1.setup(df_train, ignore_features=ignorar, target=objetivo, 
                        session_id=42, train_size=0.7, 
                        categorical_features=categoricas, numeric_features=numericas)
            # seleccionamos el mejor modelo 
            top1_exp1 = exp1.compare_models(n_select=1)
            # entrenamos el modelo 
            modelo_final = exp1.create_model(top1_exp1)
            # guardamos el modelo 
            exp1.save_model(modelo_final, 'output/my_pycaret_regression')
            # Cargamos el modelo
            my_winning_regressor = load_model('output/my_pycaret_regression')
            s = setup(data = df_test,  target = 'customer_lifetime_value')
            # Crear un modelo 
            modelo = create_model(my_winning_regressor)
